In [3]:
using DataFrames, CSV, Plots, Random, StatsBase

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1317
┌ Warning: Module Tables with build ID 1482580015134001 is missing from the cache.
│ This may mean Tables [bd369af6-aec1-5ad0-b16a-f7cc5008161c] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1008
┌ Warning: Module Tables with build ID 1482580015134001 is missing from the cache.
│ This may mean Tables [bd369af6-aec1-5ad0-b16a-f7cc5008161c] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1008
┌ Info: Skipping precompilation since __precompile__(false). Importing Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80].
└ @ Base loading.jl:1025
┌ Info: Precompiling GeometryBasics [5c1252a2-5f33-56bf-86c9-59e7332b4326]
└ @ Base loading.jl:1317
┌ Warning: Module Tables with build ID 1482580015134001 is missing from the cache.
│ This may mean Tables [bd369af6-aec1-5ad0-b16a-f7cc5008161c] does not support precompilation but is i

LoadError: ArgumentError: Package StatsBase not found in current path:
- Run `import Pkg; Pkg.add("StatsBase")` to install the StatsBase package.


# DecisionTree

In [5]:
using DecisionTree

In [6]:
data = CSV.read("Data//melb_data.csv", DataFrame);
println( "number of read data points $(size(data)[1])")
#fdata = dropmissing(data);
dropmissing!(data);
filtered = convert( Array, data[!,[:Rooms, :Bathroom, :Landsize, :BuildingArea, :YearBuilt, :Lattitude, :Longtitude]] );
y = convert( Array, data[!,:Price] );
println( "number of valid data points $(size(filtered)[1])")

number of read data points 13580
number of valid data points 6196


In [7]:
println( typeof(filtered) )
println( typeof(y) )
#first(filtered,5)
size(filtered)

Matrix{Float64}
Vector{Float64}


(6196, 7)

In [8]:
@time idx = shuffle!(collect(1:size(filtered)[1]));
sp = convert(Int, length(idx)*0.75)
trainX = filtered[idx[1:sp],:];
testX  = filtered[idx[sp+1:end],:];
trainY = y[idx[1:sp]];
testY  = y[idx[sp+1:end]];
println( "train set size ", size(trainX)[1])
println( "test  set size ", size(testX)[1])

  0.024107 seconds (84.92 k allocations: 5.586 MiB, 99.38% compilation time)
train set size 4647
test  set size 1549


In [6]:
mean_squared_error(x)  = mean( x.^2 );
mean_absolute_error(x) = mean( abs.(x) );

In [7]:
model = DecisionTreeRegressor();#max_depth=7);

In [8]:
@time DecisionTree.fit!(model, trainX, trainY)

  0.718931 seconds (2.41 M allocations: 123.172 MiB, 4.33% gc time)


DecisionTreeRegressor
max_depth:                -1
min_samples_leaf:         5
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
root:                     Decision Tree
Leaves: 749
Depth:  19

In [9]:
@time pred = DecisionTree.predict(model, testX);
println( "prediction MSE ", mean_squared_error( pred - testY ) )
println( "prediction MAE ",mean_absolute_error( pred - testY ) )

  0.110140 seconds (269.11 k allocations: 14.710 MiB, 10.61% gc time)
prediction MSE 1.412683512051058e11
prediction MAE 225596.91702941988


In [10]:
@time predT = DecisionTree.predict(model, trainX);
println( "train set prediction MSE ", mean_squared_error( predT - trainY ) )
println( "train set prediction MAE ",mean_absolute_error( predT - trainY ) )

  0.000577 seconds (4.65 k allocations: 726.266 KiB)
train set prediction MSE 5.719646014760644e10
train set prediction MAE 130855.70948828568


# Random forest

In [11]:
rf = RandomForestClassifier();

In [12]:
DecisionTree.fit!(rf, filtered, y)

RandomForestClassifier
n_trees:             10
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [131000.0, 145000.0, 170000.0, 185000.0, 200000.0, 210000.0, 216000.0, 220000.0, 222000.0, 227000.0  …  5.5e6, 5.51e6, 5.525e6, 5.6e6, 5.7e6, 6.25e6, 6.5e6, 7.65e6, 8.0e6, 9.0e6]
ensemble:            Ensemble of Decision Trees
Trees:      10
Avg Leaves: 3112.3
Avg Depth:  27.3

In [13]:
@time pred = DecisionTree.predict(rf, testX);
println( "prediction MSE ", mean_squared_error( pred - testY ) )
println( "prediction MAE ",mean_absolute_error( pred - testY ) )

  0.083479 seconds (213.34 k allocations: 11.563 MiB, 14.63% gc time)
prediction MSE 2.632691089656574e10
prediction MAE 96112.72771680655


# MLJ

In [14]:
using MLJ

# ScikitLearn

In [15]:
using ScikitLearn
using ScikitLearn.CrossValidation: train_test_split

In [16]:
trainX1, testX1, trainY1, testY1 = train_test_split(filtered, y, test_size=0.25, random_state=42);